In [2]:
import os
import re
import requests
import sys
import os
import pandas as pd
import numpy as np
import os
# importing necessary functions from dotenv library
from dotenv import load_dotenv, dotenv_values
import faiss
import numpy as np
import pymysql
import boto3
import pickle


 
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1', 
)

In [3]:
# load_dotenv(".env")


# endpoint = os.getenv('RDS_ENDPOINT')
# username = os.getenv('RDS_USERNAME')
# password = os.getenv('RDS_PASSWORD')
# database = os.getenv('RDS_DATABASE')

port = 3306

endpoint="database-colmobil.c9owiq2sebpi.us-east-1.rds.amazonaws.com"
username="admin"
password="Bb123456!"
database="databasecolmobil"
# Establish a connection to the RDS instance
connection = pymysql.connect(
    host=endpoint,
    user=username,
    password=password,
    database=database
)
cursor = connection.cursor()
# Execute a query
cursor.execute("SELECT VERSION()")
version = cursor.fetchone()
print(f"Database version: {version}")

Database version: ('8.0.39',)


In [4]:
select_sql = "SELECT car_id, additional_description, model FROM cars_collection;"
        
        # Execute the query\
cursor.execute(select_sql)

# Fetch all results
results = cursor.fetchall()
print(results)
# Create a DataFrame from the results
df_cars = pd.DataFrame(results, columns=['car_id', 'additional_description', 'model'])

((1, 'רכב שטח אידיאלי למשפחה שמחפשת את השילוב המושלם בין נוחות, עיצוב מודרני וביצועים גבוהים. עם מרווח פנימי שמספק נוחות לכל הנוסעים ויכולת תמרון מצוינת בעיר ובשטח קל, הטוסון מתאים גם לנסיעות יומיומיות וגם להרפתקאות בסוף השבוע. יתרון בולט: שילוב מושלם של ביצועים, נוחות ועיצוב שמתאים לכל דרך', 'Tucson'), (2, 'רכב חשמלי מתקדם עם עיצוב עתידני וטכנולוגיה חדשנית. אידיאלי לאורח חיים עירוני, הוא מספק ביצועים גבוהים וחוויית נהיגה שקטה וירוקה. יתרון בולט: חוויית נהיגה שקטה, מבלי להתפשר על ביצועים', ' Ioniq 5'), (3, 'רכב סדאן חשמלי עם עיצוב עתידני ומודרני, המשלב ביצועים מרשימים עם טכנולוגיה חכמה. מושלם עבור נהגים שמחפשים חווית נהיגה שקטה ונוחה, לצד תחום נהיגה ארוך שמאפשר נסיעות בין-עירוניות ללא דאגות. טעינה מהירה ויכולת חיסכון גבוהה באנרגיה הופכים אותו לאופציה מצוינת עבור כל מי שמחפש שילוב של נוחות, ביצועים ויעילות. יתרון בולט: עיצוב ייחודי, טכנולוגיה מתקדמת ויכולת ביצועים גבוהה על הכביש.', 'ioniq-6'), (4, 'רכב סופר-קומפקטי עם מנוע היברידי חסכוני המשלב ביצועים מרשימים עם חווית נהיגה דינאמית וסטי

In [5]:
import boto3
import json
 
#Create the connection to Bedrock


# Define prompt and model parameters
modelId = "amazon.titan-embed-text-v2:0"   

def generate_embeddings(prompt_data, modelId = "amazon.titan-embed-text-v2:0"):
    accept = "application/json"
    contentType = "application/json"
    sample_model_input={
        "inputText": prompt_data,
        "dimensions": 256,
        "normalize": True
    }

    body = json.dumps(sample_model_input)
    # Invoke model
    response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

    response_body = json.loads(response.get('body').read())
    embedding = response_body.get("embedding")
    # Print response and embedding
    print(f"The embedding vector has {len(embedding)} values\n{embedding[0:3]+['...']+embedding[-3:]}")
    return np.array(embedding)

# Create the combined_info column
df_recipes['combined_info'] = df_recipes.apply(
    lambda row: row['recipe_name'] + ' - ' + row['instructions_and_ingredients'],
    axis=1
)


In [6]:
df_cars['embeddings_vector_additional_description'] = df_cars['additional_description'].apply(lambda x : generate_embeddings (x))
df_cars['embeddings_vector_model'] = df_cars["model"].apply(lambda x : generate_embeddings (x))

The embedding vector has 256 values
[-0.04292425513267517, 0.024676263332366943, 0.04541261866688728, '...', -0.02125476486980915, 0.03380025923252106, 0.008139019832015038]
The embedding vector has 256 values
[-0.05778118595480919, 0.11071844398975372, 0.027333615347743034, '...', -0.02404666133224964, -0.011331340298056602, 0.007395646069198847]
The embedding vector has 256 values
[-0.10661908984184265, 0.028212517499923706, 0.030289513990283012, '...', -0.04950171709060669, 0.0536557100713253, 0.010904224589467049]
The embedding vector has 256 values
[-0.08635088801383972, 0.015401766635477543, -0.061102088540792465, '...', 0.09241059422492981, 0.09291557222604752, 0.011866934597492218]
The embedding vector has 256 values
[-0.03301191329956055, 0.004370660055428743, -0.027542483061552048, '...', -0.072665274143219, 0.008497148752212524, 0.0033451418858021498]
The embedding vector has 256 values
[-0.06422393023967743, 0.05481971055269241, 0.04403926804661751, '...', 0.096335895359516

In [7]:
df_cars['embeddings_vector_additional_description']

0     [-0.04292425513267517, 0.024676263332366943, 0...
1     [-0.05778118595480919, 0.11071844398975372, 0....
2     [-0.10661908984184265, 0.028212517499923706, 0...
3     [-0.08635088801383972, 0.015401766635477543, -...
4     [-0.03301191329956055, 0.004370660055428743, -...
5     [-0.06422393023967743, 0.05481971055269241, 0....
6     [-0.09265321493148804, -0.008034119382500648, ...
7     [-0.03558187559247017, 0.04594363272190094, 0....
8     [-0.1762755960226059, -0.034944500774145126, -...
9     [-0.06025490537285805, 0.09480667114257812, 0....
10    [-0.11116329580545425, 0.07804117351770401, -0...
11    [-0.1464223861694336, 0.06627539545297623, -0....
12    [-0.045726824551820755, 0.11597101390361786, 0...
13    [-0.061537906527519226, 0.10069838911294937, 0...
14    [-0.12182297557592392, 0.06885646283626556, -0...
15    [-0.06475123763084412, 0.07441026717424393, -0...
16    [-0.03775367513298988, 0.03372173011302948, -0...
17    [-0.06445290148258209, 0.04140487313270569

In [8]:
def saving_embedding_data_cars(id,embeddings_vector_additional_description, embeddings_vector_model):
        
        # Convert the embedding vector to bytes
        embeddings_vector_additional_description_bytes = pickle.dumps(embeddings_vector_additional_description)
        embedding_vector_model_bytes = pickle.dumps(embeddings_vector_model)
        
        # Define the SQL update statement
        update_sql = f"""
        UPDATE cars_collection
        SET embeddings_vector_additional_description = %s,
        embeddings_vector_model = %s        
        WHERE car_id = %s
        """
        
        # Execute the update statement (replace `1` with the appropriate row ID)
        cursor.execute(update_sql, (embeddings_vector_additional_description_bytes,embedding_vector_model_bytes, id))
        
        # Commit the changes
        connection.commit()

In [9]:
for id in df_cars['car_id']:
    embeddings_vector_additional_description = df_cars.loc[df_cars['car_id'] == id].embeddings_vector_additional_description
    embedding_vector_model = df_cars.loc[df_cars['car_id'] == id].embeddings_vector_model
    saving_embedding_data_cars(id,embeddings_vector_additional_description, embedding_vector_model)

In [10]:
select_sql = f"SELECT car_id, embeddings_vector_additional_description, embeddings_vector_model FROM cars_collection;"

# Execute the query
cursor.execute(select_sql)

# Fetch all results
results = cursor.fetchall()

In [11]:
results

((1,
  b'\x80\x04\x95\xf4\n\x00\x00\x00\x00\x00\x00\x8c\x12pandas.core.series\x94\x8c\x06Series\x94\x93\x94)\x81\x94}\x94(\x8c\x04_mgr\x94\x8c\x1epandas.core.internals.managers\x94\x8c\x12SingleBlockManager\x94\x93\x94)\x81\x94(]\x94\x8c\x18pandas.core.indexes.base\x94\x8c\n_new_Index\x94\x93\x94h\x0b\x8c\x05Index\x94\x93\x94}\x94(\x8c\x04data\x94\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x01\x85\x94h\x15\x8c\x05dtype\x94\x93\x94\x8c\x02i8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89C\x08\x00\x00\x00\x00\x00\x00\x00\x00\x94t\x94b\x8c\x04name\x94Nu\x86\x94R\x94a]\x94h\x14h\x17K\x00\x85\x94h\x19\x87\x94R\x94(K\x01K\x01\x85\x94h\x1e\x8c\x02O8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01|\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?t\x94b\x89]\x94h\x14h\x17K\x00\x85\x94h\x19\x87\x94R\x94(K\x01M\x00\x01\x85\x94h\x1e\x8c\x02f8\x94\x89\x88\x8

In [12]:

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()